# Imports

In [17]:
import yaml
import re
import numpy as np

# Metrics
from scipy.spatial.distance import directed_hausdorff
from scipy.spatial.distance import euclidean
from scipy.spatial.transform import Rotation

from fastdtw import fastdtw

# Visualization
import plotly.graph_objects as go

# Ros modules
import rosbag2_py
import rclpy.serialization
from geometry_msgs.msg import PoseStamped, PoseWithCovarianceStamped

# Data generation

## Build the project

In [18]:
# %cd ../..
# ! colcon build

## Run nodes and save the outputs to .txt files

In [19]:
# ! ros2 bag record -o calculated_pose /new_pose
# ! ros2 bag record -o original_pose /clean_pose

# Extract position sequence

## Change working directory for the Notebook to read the recordings

In [20]:
%cd ..
%cd recordings/

/home/pablo-pc/GITHUB/TFG/project_ws/src
/home/pablo-pc/GITHUB/TFG/project_ws/src/recordings


/home/pablo-pc/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning:

This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.



## Save the 3D paths

### Saving the original path

In [21]:
 # Initialize the data that will be published
def read_bag(base_path):
       storage_options = rosbag2_py.StorageOptions(uri=base_path, storage_id="sqlite3")
       converter_options = rosbag2_py.ConverterOptions(input_serialization_format = "cdr", output_serialization_format = "cdr")
       reader = rosbag2_py.SequentialReader()
       reader.open(storage_options, converter_options)
       return reader
        

In [22]:
original_path = []

bag_original = read_bag("pose/original/original_pose/original_pose.db3")

while bag_original.has_next():
    
    # Read the next message in the queue
    topic, msg, t = bag_original.read_next()

    # Deserialize the msg to extract the x, y and z values
    deserialized_msg = rclpy.serialization.deserialize_message(msg, PoseWithCovarianceStamped)

    original_path.append([deserialized_msg.pose.pose.position.x,
                          deserialized_msg.pose.pose.position.y,
                          deserialized_msg.pose.pose.position.z])
    
original_path = np.array(original_path)

[INFO] [1756936792.806075671] [rosbag2_storage]: Opened database 'pose/original/original_pose/original_pose.db3' for READ_ONLY.


### Saving the path calculated from the clean_pcl input

In [23]:
clean_path = []

bag_clean = read_bag("pose/clean_pcl/clean_pcl_pose/clean_pcl_pose.db3")

while bag_clean.has_next():
    
    # Read the next message in the queue
    topic, msg, t = bag_clean.read_next()

    # Deserialize the msg to extract the x, y and z values
    deserialized_msg = rclpy.serialization.deserialize_message(msg, PoseStamped)

    clean_path.append([deserialized_msg.pose.position.x,
                            deserialized_msg.pose.position.y,
                            deserialized_msg.pose.position.z])
    
clean_path = np.array(clean_path)

[INFO] [1756936792.884447929] [rosbag2_storage]: Opened database 'pose/clean_pcl/clean_pcl_pose/clean_pcl_pose.db3' for READ_ONLY.


### Saving the path calculated from the noisy_pcl input

In [24]:

noisy_path = []

bag_noisy = read_bag("pose/noisy_pcl/noisy_pcl_pose/noisy_pcl_pose.db3")

while bag_noisy.has_next():
    
    # Read the next message in the queue
    topic, msg, t = bag_noisy.read_next()

    # Deserialize the msg to extract the x, y and z values
    deserialized_msg = rclpy.serialization.deserialize_message(msg, PoseStamped)

    noisy_path.append([deserialized_msg.pose.position.x,
                            deserialized_msg.pose.position.y,
                            deserialized_msg.pose.position.z])
    
noisy_path = np.array(noisy_path)

[INFO] [1756936792.919332122] [rosbag2_storage]: Opened database 'pose/noisy_pcl/noisy_pcl_pose/noisy_pcl_pose.db3' for READ_ONLY.


# Metricas y visualización

## Metricas

### Absolut Trayectory Error (ATE)

In [25]:
def align_trajectories(traj_ref, traj_est):

    # Alings the estimate trayectory to the reference using a Sim(3) transformation

    # Centroids calculation
    centroid_ref = np.mean(traj_ref, axis=0)
    centroid_est = np.mean(traj_est, axis=0)
    
    # Center data
    traj_ref_centered = traj_ref - centroid_ref
    traj_est_centered = traj_est - centroid_est

    # Covariance matrix
    H = traj_est_centered.T @ traj_ref_centered

    # SVD decomposition for optimum rotation search
    U, _, Vt = np.linalg.svd(H)
    
    # Rotation matrix
    R = Vt.T @ U.T 

    # Valid rotation matrix check
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = Vt.T @ U.T

    # Apply rotation and translation
    traj_est_aligned = (traj_est_centered @ R.T) + centroid_ref

    return traj_est_aligned

def compute_ate(traj_ref, traj_est):
    # Compute Absolut Trayectory error between two paths
    
    # Aling estimated trayectories
    traj_est_aligned = align_trajectories(traj_ref, traj_est)

    # Compute point to point error
    errors = np.linalg.norm(traj_ref - traj_est_aligned, axis=1)

    # Compute metrics
    ate_mean = np.mean(errors)
    ate_std = np.std(errors)

    return ate_mean, ate_std, errors

## ATE

### Original - CleanPCL

In [26]:
ate_mean, ate_std, errors = compute_ate(original_path, clean_path)

print("ATE Mean: " + str(ate_mean)+ " meters")
print("ATE STD: " + str(ate_std) + " meters")
print(f"Max - Min error: {np.max(errors)} - {np.min(errors)}")

ATE Mean: 0.3217524462271145 meters
ATE STD: 0.12168893655068846 meters
Max - Min error: 0.6175477542389056 - 0.09460674212877905


### Original - NoisyPCL

In [27]:
ate_mean, ate_std, errors = compute_ate(original_path, noisy_path)

print("ATE Mean: " + str(ate_mean)+ " meters")
print("ATE STD: " + str(ate_std) + " meters")
print(f"Max - Min error: {np.max(errors)} - {np.min(errors)}")

ATE Mean: 0.327063310851359 meters
ATE STD: 0.14548091092795593 meters
Max - Min error: 0.6152245606829154 - 0.040570651906988386


## Hausdorff

### Original - CleanPCL

In [28]:
d1_oc = directed_hausdorff(original_path, clean_path)[0]
d2_co = directed_hausdorff(clean_path, original_path)[0]

hausdorff_distance = max(d1_oc, d2_co)
print("Hausdorff Distance:", hausdorff_distance)

Hausdorff Distance: 1.1022694627008474


### Original - NoisyPCL

In [29]:
d1_on = directed_hausdorff(original_path, noisy_path)[0]
d2_no = directed_hausdorff(noisy_path, original_path)[0]

hausdorff_distance = max(d1_on, d2_no)
print("Hausdorff Distance:", hausdorff_distance)

Hausdorff Distance: 1.2890047623696028


## Frechet-like Distance (DTW)

### Original - CleanPCL

In [30]:
distance_oc, _ = fastdtw(clean_path, original_path, dist=euclidean)
print("Frechet-like Distance (DTW):", distance_oc)

Frechet-like Distance (DTW): 468.9198338195788


### Original - NoisyPCL

In [31]:
distance_on, _ = fastdtw(noisy_path, original_path, dist=euclidean)
print("Frechet-like Distance (DTW):", distance_on)

Frechet-like Distance (DTW): 499.1673830413068


## Visualización

In [32]:
# Create 3D scatter lines
fig = go.Figure()

# Adding Original Path to figure
fig.add_trace(go.Scatter3d(
    x=original_path[:,0], y=original_path[:,1], z=original_path[:,2],
    mode='lines+markers', marker=dict(size=4, color='green'),
    name="Original"
))

# Adding Calculated Path to figure
fig.add_trace(go.Scatter3d(
    x=clean_path[:,0], y=clean_path[:,1], z=clean_path[:,2],
    mode='lines+markers', marker=dict(size=4, color='yellow'),
    name="Clean"
))

# Adding Calculated Path to figure
fig.add_trace(go.Scatter3d(
    x=noisy_path[:,0], y=noisy_path[:,1], z=noisy_path[:,2],
    mode='lines+markers', marker=dict(size=4, color='red'),
    name="Noisy"
))

# Labels & Layout
fig.update_layout(
    title="Interactive 3D Path Visualization",
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z"
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# Show the figure
fig.show()